<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Designate-a-Set-of-Movies" data-toc-modified-id="Designate-a-Set-of-Movies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Designate a Set of Movies</a></span></li><li><span><a href="#Get-the-Set-of-Characters" data-toc-modified-id="Get-the-Set-of-Characters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get the Set of Characters</a></span></li><li><span><a href="#Initialize-Notebook-State" data-toc-modified-id="Initialize-Notebook-State-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Initialize Notebook State</a></span></li><li><span><a href="#Do-Clustering-Pass" data-toc-modified-id="Do-Clustering-Pass-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Do Clustering Pass</a></span></li><li><span><a href="#Do-Search-Pass" data-toc-modified-id="Do-Search-Pass-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Do Search Pass</a></span><ul class="toc-item"><li><span><a href="#Save-the-Results" data-toc-modified-id="Save-the-Results-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Save the Results</a></span></li></ul></li></ul></div>

<b>Imports and Setup. Run the initialization cells.<b>

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
import datetime
import itertools
import io
import os
import time
import math
import random
import numpy as np
np.warnings.filterwarnings('ignore')
import pickle
from collections import defaultdict
from django.db import transaction

from esper.stdlib import *
from esper.prelude import *
import esper.face_embeddings as face_embeddings

NUM_PER_PAGE = 25

def flatten(l):
    return [item for sublist in l for item in sublist]

def query_faces(ids):
    faces = Face.objects.filter(id__in=ids)
    return faces.values(
        'id', 'bbox_y1', 'bbox_y2', 'bbox_x1', 'bbox_x2',
        'frame__number', 'frame__video__id', 'frame__video__fps',
        'shot__min_frame', 'shot__max_frame')

def query_sample(qs, n):
    return qs.order_by('?')[:n]

def query_faces_result(faces, expand_bbox=0.05):
    """Replaces qs_to_result"""
    result = []
    for face in faces:
        min_frame = face['frame__number']
        face_result = {
            'type': 'flat', 'label': '', 
            'elements': [{
                'objects': [{
                    'id': face['id'],
                    'background': False,
                    'type': 'bbox',
                    'bbox_y1': max(face['bbox_y1'] - expand_bbox, 0),
                    'bbox_y2': min(face['bbox_y2'] + expand_bbox, 1),
                    'bbox_x1': max(face['bbox_x1'] - expand_bbox, 0),
                    'bbox_x2': min(face['bbox_x2'] + expand_bbox, 1),
                }], 
                'min_frame': min_frame,
                'video': face['frame__video__id']
            }]
        }
        result.append(face_result)
    return {'type': 'Face', 'count': 0, 'result': result}

def sort_ids_by_distance(ids, embs):
    dists = face_embeddings.dist(ids, targets=embs)
    return [i for _, i in sorted(zip(dists, ids))]

def sort_faces_by_distance(faces, embs, ascending=False):
    ids = [f['id'] for f in faces]
    id_to_dist = {
        k: v for k, v in zip(ids, face_embeddings.dist(ids, targets=embs))
    }
    order_const = 1 if ascending else -1
    faces.sort(key=lambda x: order_const * id_to_dist[x['id']])
    return faces

def get_clusters(face_ids, k):
    clusters = [[] for _ in range(k)]
    for i, c in face_embeddings.kmeans(face_ids, k):
        clusters[c].append(i)
    clusters.sort(key=lambda x: -len(x))
    return clusters

def get_clusters_recursive(face_ids, max_size=1000):
    clusters = []
    branch = math.ceil(len(face_ids) / max_size)
    for c in get_clusters(face_ids, branch):
        if len(c) > max_size:
            clusters.extend(get_clusters_recursive(c, max_size))
        else:
            clusters.append(c)
    clusters.sort(key=lambda x: -len(x))
    return clusters

def get_faces(video):
    face_ids = [f['id']for f in Face.objects.filter(frame__video_id=video.id).values('id')]
    print('Selected film contains {} faces'.format(len(face_ids)))
    embs_exist = face_embeddings.exists(face_ids)
    if not all(embs_exist):
        print('Missing {} face embeddings'.format(len(face_ids) - sum(embs_exist)),
              file=sys.stderr)
    face_dict = {f['id']: f for f in query_faces(face_ids)}
    assert len(face_ids) == len(face_dict)
    return face_ids, face_dict

def parse_identity_list(text):
    identities = set()
    for line in text.split('\n'):
        line = line.strip()
        if len(line) == 0:
            continue
        name, character = line.lower().split(',', 1)
        name = name.strip()
        character = character.strip()
        if len(name) == 0:
            raise ValueError('Name cannot be empty')
        if len(character) == 0:
            raise ValueError('Character cannot be empty')
        identities.add((name, character))
    if len(identities) == 0:
        raise ValueError('No identities specified')
    return identities

def exclude_labeled_faces(face_ids):
    exclude_set = {
        f['face__id'] for f in FaceIdentity.objects.filter(
            face__id__in=face_ids
        ).distinct('face__id').values('face__id')
    }
    return [i for i in face_ids if i not in exclude_set]

def show_people_textbox():
    people_textbox = widgets.Textarea(
        value='',
        layout=widgets.Layout(width='auto', height='400px'),
        style={'description_width': 'initial'},
        placeholder='e.g., Daniel Radclife, Harry Potter',
        description='<b>People:</b> name, identity (1 per line)',
        disabled=False
    )
    valid_checkbox = widgets.Valid(
        value=False,
        style={'description_width': 'initial'},
        description='<b>Valid?</b>',
    )
    def update(b):
        global people
        try:
            people = parse_identity_list(people_textbox.value)
            valid_checkbox.value = True
        except:
            people = []
            valid_checkbox.value = False
    people_textbox.observe(update, names='value')
    display(people_textbox)
    display(valid_checkbox)

def load_characters(video):
    character_actors = CharacterActor.objects.filter(video=video).annotate(
        character_name=F('character__name'),
        actor_name=F('actor__name')
    ).all()
    global people
    
    print('These are the main characters in the movie:')
    people = []
    for ca in character_actors:
        people.append((ca.actor_name, ca.character_name, ca.id))
        if ca.primary:
            print(ca.actor_name, ',', ca.character_name)
    
    print()
    print('Here are all the characters available to label in this movie:')
    for ca in character_actors:
        print(ca.actor_name, ',', ca.character_name)
    
def format_identity(person, character, character_actor_id):
    return '{} :: {}'.format(person, character)

def get_identity_value(person, character, character_actor_id):
    return (format_identity(person, character, character_actor_id), character_actor_id)

def parse_identity(s):
    return tuple(s.split(' :: '))

def get_identity_options():
    return [('', -1)] + [
        (format_identity(*x), get_identity_value(*x))
        for x in people]

def get_searchable_identity_dropdown():
    identity_dropdown = widgets.Dropdown(
        options=get_identity_options(),
        value=-1,
        description='Person:',
        disabled=False
    )
    identity_text = widgets.Text(
        value='',
        placeholder='search...',
        continuous_update=True,
        disabled=False
    )
    def on_update(b):
        identity_options = get_identity_options()
        identity_dropdown.options = identity_options
        search_value = identity_text.value.strip().lower()
        for opt in identity_options:
            if search_value in opt[0]:
                identity_dropdown.value = opt[1]
                break
        else:
            identity_dropdown.value = identity_options[0][0]
    identity_text.observe(on_update, names='value')
    return identity_text, identity_dropdown

def int_prompt(msg, min_val, max_val, default):
    line = input('{}, range=[{}, {}], default={}: '.format(
                 msg, min_val, max_val, default)).strip()
    if line == '':
        value = default
    else:
        value = int(line)
    if value < min_val or value > max_val:
        raise ValueError('Out of range.')
    return value

def prepare_orm_objects(person_to_clusters):
    # Need 2 labelers to get around unique constraint
    time_str = datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
    labeler_name = 'cluster-{}'.format(time_str)
    labeler = Labeler.objects.create(name=labeler_name)
    print('Labeler created:', labeler_name)
    
    face_identities = []
    for identity, clusters in person_to_clusters.items():
        cluster_face_ids = set(flatten(clusters))
        if len(cluster_face_ids) == 0:
            print('Skipping: no faces for {}'.format(identity[0]), file=sys.stderr)
            continue

        names, characteractor_id = identity

        for i in cluster_face_ids:
            face_identities.append(FaceCharacterActor(
                labeler=labeler,
                face_id=i,
                probability=1.,
                characteractor_id=characteractor_id
            ))
    return labeler, face_identities

def save_orm_objects(face_identities):
    print('Commiting {} face identity labels'.format(len(face_identities)))
    with transaction.atomic():
        FaceCharacterActor.objects.bulk_create(face_identities)
    print('Done!')

def print_videos(video):
    print('You selected this video:')
    print('  ', video.name)
    
# def print_videos_with_faces():
#     print('The following videos have face features:')
#     key = 'face__frame__video__name'
#     for f in FaceFeatures.objects.distinct(key).values(key):
#         print('  ', f[key])
        
#print_videos_with_faces()
print("Done with initialization!")

# Designate a Set of Movies

`video` needs to be a single Django Video object.

In [ ]:
video = Video.objects.get(name='apollo 13')

In [ ]:
print_videos(video)

# Get the Set of Characters

Query the database to get all the characters. We have already scraped which characters are main characters.

In [ ]:
#show_people_textbox()
load_characters(video)

# Initialize Notebook State

Global variables to track labeling state.

In [ ]:
person_to_clusters = defaultdict(list)
unassigned_face_ids, face_dict = get_faces(video)
initial_num_faces = len(unassigned_face_ids)

<b>Optional: filter to only unlabeled faces</b>

In [ ]:
unassigned_face_ids = exclude_labeled_faces(unassigned_face_ids)

# Do Clustering Pass

The idea of this section is that by clustering across all of the unassigned faces and making assignments, main characters will be captured. `do_clustering_pass()` uses k-means clustering to compute clusters of the maximum size specified.

When evaluating the clusters, the options are to accept, ignore (removes the faces from being clustered), or split (returns the faces to unassigned). You add more names, just be sure to refresh the list of people for the person to appear. 

Pressing "finish clustering pass" will return any unselected clusters to the unassigned pool, while committing your other selections. After you click this, some stats will be printed.

You can re-run the cell to do additional clustering passes.

<b>To expand a frame, hover and press '='. Press '=' again to shrink it.</b> 

<b>There is a hidden initialization cell below.<b>

In [ ]:
SUB_CLUSTER_THRESH = 100
SUB_CLUSTER_COUNT = 5

def visualize_cluster(cluster_id, face_ids, ignored_ids, unassigned_ids, 
                      clusters_done, sort='random'):
    print('Cluster has {} faces'.format(len(face_ids)))
    if sort != 'random':
        if len(face_ids) > SUB_CLUSTER_THRESH:
            sub_clusters = get_clusters(face_ids, SUB_CLUSTER_COUNT)
        else:
            sub_clusters = face_ids
        mean_embs = [face_embeddings.mean(c) for c in sub_clusters]
        face_ids = sort_ids_by_distance(face_ids, mean_embs)
        if sort == 'descending':
            face_ids = face_ids[::-1]
    else:
        random.shuffle(face_ids)
    faces = [face_dict[i] for i in face_ids]

    sort_button = widgets.ToggleButtons(
        options=['random', 'ascending', 'descending'],
        value=sort,
         style={'description_width': 'initial'},
        description='Sort (by L2-distance to centers):',
        disabled=False,
        button_style=''
    )
    def refresh(b):
        new_sort = sort_button.value
        clear_output()
        visualize_cluster(cluster_id, face_ids, ignored_ids, unassigned_ids, 
                          clusters_done, sort=new_sort)
    sort_button.observe(refresh, names='value')
    display(sort_button)
    
    identity_text, identity_dropdown = get_searchable_identity_dropdown()
    
    accept_button = widgets.Button(description='Accept cluster', button_style='success')
    def on_accept(b):
        person = identity_dropdown.value
        label = identity_dropdown.label
        clear_output()
        if person[1] == -1:
            print('No person specified. Cluster ignored.', file=sys.stderr)
            ignored_ids.update(face_ids)
        else:
            person_to_clusters[person].append(face_ids)
            print('Assigned cluster to {} ({} faces).'.format(label, len(face_ids)))
        clusters_done.add(cluster_id)
    accept_button.on_click(on_accept)
    ignore_button = widgets.Button(description='Ignore cluster', button_style='danger')
    def on_ignore(b):
        ignored_ids.update(face_ids)
        clusters_done.add(cluster_id)
        clear_output()
        print('Ignored cluster ({} faces).'.format(len(face_ids)))
    ignore_button.on_click(on_ignore)
    reject_button = widgets.Button(description='Split cluster', button_style='warning')
    def on_reject(b):
        unassigned_ids.update(face_ids)
        clusters_done.add(cluster_id)
        clear_output()
        print('Returned cluster to unassigned set ({} faces).'.format(len(face_ids)))
    reject_button.on_click(on_reject)
    
    cluster_widget = esper_widget(
        query_faces_result(faces), results_per_page=NUM_PER_PAGE, 
        crop_bboxes=True, jupyter_keybindings=True, disable_playback=True)
    display(widgets.HBox([accept_button, ignore_button, reject_button,
                          identity_dropdown, identity_text]))
    display(cluster_widget)
    
def do_clustering_pass(ids, branch=10):
    print('Clustering {} faces.'.format(len(ids)))
    default_size = math.ceil(len(ids) / branch / 100) * 100
    cluster_size = int_prompt('Specify a maximum cluster size',
                              100, max(default_size, 5000), default_size)
    
    clusters = get_clusters_recursive(ids, max_size=cluster_size)
    print('Found {} clusters. (Ordered from largest to smallest).'.format(len(clusters)))
    
    unassigned_ids = set()
    ignored_ids = set()
    clusters_done = set()
    
    outputs = [widgets.Output() for _ in range(len(clusters))]
    tabs = widgets.Tab(children=outputs)
    for i in range(len(clusters)):
        tabs.set_title(i, str(i))
    tabs_loaded = set()
    def load_current_tab(b):
        i = tabs.selected_index
        if not i in tabs_loaded:
            tabs_loaded.add(i)
        with outputs[i]:
            visualize_cluster(i, clusters[i], ignored_ids, unassigned_ids,
                              clusters_done)
    tabs.observe(load_current_tab, names='selected_index')
    load_current_tab(None)
    
    finish_button = widgets.Button(description='Finish clustering pass',
                                   button_style='success')
    def on_finish(b):
        clear_output()
        if not len(clusters_done) == len(clusters):
            remaining_clusters = set(range(len(clusters))) - clusters_done
            print('Not all clusters selected... (treating them as unassigned)')
            for c in remaining_clusters:
                unassigned_ids.update(clusters[c])
        finish_button.disabled = True
        assigned_count = len(ids) - len(ignored_ids) - len(unassigned_ids)
        print('Clustering pass statistics:')
        print('  ', 'Assigned: {}'.format(assigned_count))
        print('  ', 'Ignored: {}'.format(len(ignored_ids)))
        print('  ', 'Unassigned: {}'.format(len(unassigned_ids)))
        global unassigned_face_ids
        unassigned_face_ids = list(unassigned_ids)
        
        print()
        print_assignment_state()
    finish_button.on_click(on_finish)
    display(tabs)
    display(finish_button)
    
def print_assignment_state():
    print('{} faces are unassigned (of {} initially)'.format(
          len(unassigned_face_ids), initial_num_faces))
    if len(person_to_clusters) > 0:
        print('The following people have faces assigned:')
        for k, v in person_to_clusters.items():
            id_set = set()
            for c in v:
                id_set.update(c)
            print('  {}: {} faces'.format(k[0], len(id_set)))
    else:
        print('No people have faces assigned.')
        
print('Initialization code succeeded')

In [ ]:
print_assignment_state()

In [ ]:
do_clustering_pass(unassigned_face_ids)

# Do Search Pass

This is useful for selecting specific unclustered people. The idea is to find 1 or more examples of a person, and then to sort all the unassigned faces by distance to these examples.

First, you will be shown unassigned faces at random. Select 1 or more examples of the person you want to label from this set. <b>To select an example, press '['.</b> Hit submit when you are ready to move on to the next step.

Next, you will be shown the remaining unassigned faces in order of ascending distance. Select all the faces that are of this person. <b>To select a page of examples, press '{' (i.e., 'shift + ['). To deselect a face, press '[' while hovering over it.</b> Hit submit to finish the assignment.

<b>There is a hidden initialization cell below.<b>

In [ ]:
def do_search_pass(ids):
    faces = [face_dict[i] for i in ids]
    random.shuffle(faces)
    
    identity_text, identity_dropdown = get_searchable_identity_dropdown()
    
    def do_selection_phase(faces, example_embs):
        faces = sort_faces_by_distance(faces, example_embs, ascending=True)

        select_widget = esper_widget(
            query_faces_result(faces), results_per_page= 4 * NUM_PER_PAGE, 
            crop_bboxes=True, jupyter_keybindings=True, disable_playback=True)

        submit_button = widgets.Button(description='Submit selections', 
                                       button_style='success')
            
        def on_submit(b):
            selected_idxs = select_widget.selected
            selected_ids = [faces[i]['id'] for i in selected_idxs]
            person = identity_dropdown.value
            if person[1] == -1:
                print('No person selected... Try again.', file=sys.stderr)
                return

            clear_output()
            person_to_clusters[person].append(selected_ids)
            print('Assigned {} faces to {}.'.format(len(selected_ids), person))

            global unassigned_face_ids
            unassigned_face_ids = list(set(ids) - set(selected_ids))

            print()
            print_assignment_state()
        submit_button.on_click(on_submit)

        print('Select faces matching the target person')
        display(widgets.HBox([submit_button, identity_dropdown, identity_text]))
        display(select_widget)
    
    # Submit examples from random faces
    example_widget = esper_widget(
        query_faces_result(faces), results_per_page=NUM_PER_PAGE, 
        crop_bboxes=True, jupyter_keybindings=True, disable_playback=True)
    submit_example_button = widgets.Button(description='Submit examples', 
                                           button_style='success')
    def on_example_submit(b):
        example_idxs = example_widget.selected
        clear_output()
        if len(example_idxs) == 0:
            print('No examples selected. Aborting.', file=sys.stderr)
            return
        else:
            print('Selected {} examples. Ordering unassigned faces by ascending distance.'.format(
                  len(example_idxs)))
            print()
        
        example_ids = [faces[i]['id'] for i in example_idxs]
        example_embs = [x for _, x in face_embeddings.get(example_ids)]
        do_selection_phase(faces, example_embs)
    submit_example_button.on_click(on_example_submit)
    
    # Submit examples from existing clusters
    submit_existing_button = widgets.Button(description='Submit name', 
                                            button_style='success')
    def on_existing_submit(b):
        person = identity_dropdown.value
        if person[1] == -1:
            print('No person selected... Try again.', file=sys.stderr)
            return
        if person not in person_to_clusters:
            print('No clusters for person selected... Try someone else.', 
                  file=sys.stderr)
            return
        person_example_ids = set(flatten(person_to_clusters[person]))
        if len(person_example_ids) == 0:
            print('No examples for person selected... Try someone else.', 
                  file=sys.stderr)
            return
        clear_output()
        mean_emb = face_embeddings.mean(list(person_example_ids))
        do_selection_phase(faces, [mean_emb])
    submit_existing_button.on_click(on_existing_submit)
    
    print('Select face search examples:')
    accordion = widgets.Accordion(children=[
        widgets.HBox([identity_dropdown, identity_text, submit_existing_button]),
        widgets.VBox([submit_example_button, example_widget])
    ])
    accordion.set_title(0, 'Choose examples from existing clusters')
    accordion.set_title(1, 'Choose examples from unassigned faces')
    display(accordion)
    
print('Initialization code succeeded')

In [ ]:
print_assignment_state()

In [ ]:
do_search_pass(unassigned_face_ids)

## Save the Results

This will save your labels to the database. `prepare_orm_objects()` will create labeler rows, tied to this session. The new face identity rows, corresponding to your assignments, will have foreign keys to these labelers.

`save_orm_objects()` will save the identity labels to the database.

In [ ]:
print_assignment_state()

In [ ]:
labelers, face_identities = prepare_orm_objects(person_to_clusters)

In [ ]:
save_orm_objects(face_identities)

<i>Help! I messed up and committed bad labels... What should I do?</i> Hopefully, if the notebook is still running, the labelers are still defined. Then, you can run the following.

In [ ]:
# for labeler in labelers:
#     display(labeler.delete())